# CIFAR Conv Net

И так, в этом ноутбуке Вы сделаете превую в своей жизни сверточную сеть! На сложном датасете. Cкачайте его кстати, 

In [47]:
!mkdir cifar10
!curl -o cifar-10-python.tar.gz https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xvzf cifar-10-python.tar.gz -C cifar10

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  162M  100  162M    0     0  3593k      0  0:00:46  0:00:46 --:--:-- 4861k
cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [26]:
import numpy as np
#import matplotlib.pyplot as plt

#%matplotlib inline

In [27]:
from cifar import load_CIFAR10
#plt.rcParams['figure.figsize'] = (10.0, 8.0) 

cifar10_dir = './cifar10/cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

In [ ]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8').transpose(1, 2, 0))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

<h1 align="center">First of all -- Checking Questions</h1> 

**Вопрос 1**: Чем отличаются современные сверточные сети от сетей 5 летней давности?

<Ответ>

**Вопрос 2**: Какие неприятности могут возникнуть во время обучения современных нейросетей?

<Ответ>

**Вопрос 3**: У вас есть очень маленький датасет из 100 картинок, но вы очень хотите использовать нейросеть, какие неприятности вас ждут и как их решить?

<Ответ>

**Вопрос 4**: У вас есть очень маленький датасет из 100 картинок, классификация, но вы очень хотите использовать нейросеть, какие неприятности вас ждут и как их решить? что делать если первый вариант  решения не заработает?

<Ответ>

**Вопрос 5**: Как сделать стайл трансфер для музыки? oO

<Ответ>

In [28]:
import lasagne
from theano import tensor as T
from lasagne.nonlinearities import *

input_X = T.tensor4("X")
target_y = T.vector("target Y integer",dtype='int32')

### Соберите нейронку: 
- Many times x (Conv+Pool)
- Many small convolutions like 3x3
- Batch Norm 
- Residual Connection
- Data Augmentation 
- Learning rate Schedule 
- ...

### Для вдохновения 
- http://torch.ch/blog/2015/07/30/cifar.html
- http://www.robots.ox.ac.uk/~vgg/research/very_deep/
- https://www.cs.unc.edu/~wliu/papers/GoogLeNet.pdf
- https://github.com/szagoruyko/wide-residual-networks 

### Самое интересное
- Для сдачи задания нужно набрать на точность тесте > **92.5**% (это займет много времени, торопитесь :) )
- Для получения бонусных баллов > **95.0**%
- Будет очень хорошо если вы придумаете свою архитектуру или сможете обучить что-то из вышеперечисленного :)
- А для обучения всего этого добра вам будет куда удобнее использовать GPU на Amazon 
    - Инструкция https://github.com/persiyanov/ml-mipt/tree/master/amazon-howto 
    - Вам помогут tmux, CuDNN, ssh tunnel, nvidia-smi, ... 
    - Wish you get fun :)

In [29]:
from lasagne.layers import batch_norm

In [30]:
def normal(x):
    x = x-128
    x = x/128.
    return x

In [31]:
X_train = normal(X_train)
X_test = normal(X_test)

In [32]:
net = lasagne.layers.InputLayer(shape=(None, 3, 32, 32), input_var=input_X) 

net =lasagne.layers.Conv2DLayer(net, num_filters=32, filter_size=(3,3), nonlinearity=rectify, 
                                W=lasagne.init.HeNormal(gain='relu')) 
net = lasagne.layers.Conv2DLayer(net, num_filters=32, filter_size=3) 

net = lasagne.layers.MaxPool2DLayer(net, pool_size=(2,2)) 

net = lasagne.layers.Conv2DLayer(net, num_filters=64, filter_size=(3,3)) 
net = lasagne.layers.Conv2DLayer(net, num_filters=64, filter_size=3) 

net = lasagne.layers.MaxPool2DLayer(net, pool_size=(2,2))

net = lasagne.layers.DenseLayer(net, num_units=128) 
net = lasagne.layers.DropoutLayer(net, p=0.3) 
net = lasagne.layers.DenseLayer(net, num_units=64)
net = lasagne.layers.DropoutLayer(net, p=0.3) 

net = lasagne.layers.DenseLayer(net, num_units = 10, nonlinearity=softmax)

In [33]:
y_predicted = lasagne.layers.get_output(net)
all_weights = lasagne.layers.get_all_params(net, trainable=True)
print all_weights

[W, b, W, b, W, b, W, b, W, b, W, b, W, b]


In [34]:
#l2_reg = lasagne.regularization.regularize_network_params(net, penalty=lasagne.regularization.l2)
loss = lasagne.objectives.categorical_crossentropy(y_predicted, target_y).mean()
#loss = loss+l2_reg
accuracy = lasagne.objectives.categorical_accuracy(y_predicted, target_y).mean()

In [35]:
update = lasagne.updates.adam(loss, params=all_weights)
train_fun = theano.function([input_X,target_y],[loss, accuracy], updates=update, allow_input_downcast=True)
accuracy_fun = theano.function([input_X,target_y],accuracy, allow_input_downcast=True)

### Вот и всё, пошли её учить

In [36]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

# Процесс обучения

In [37]:
import time
from tqdm import tqdm

num_epochs = 20 #количество проходов по данным

batch_size = 50 #размер мини-батча

for epoch in tqdm(range(num_epochs)):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batch_size, shuffle=True):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_test, y_test, batch_size, shuffle=True):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(val_acc / val_batches * 100))

  5%|▌         | 1/20 [01:09<21:52, 69.08s/it]

Epoch 1 of 20 took 69.083s
  training loss (in-iteration):		1.584583
  train accuracy:		41.82 %
  validation accuracy:		53.89 %


 10%|█         | 2/20 [02:18<20:43, 69.06s/it]

Epoch 2 of 20 took 69.012s
  training loss (in-iteration):		1.163750
  train accuracy:		58.77 %
  validation accuracy:		61.47 %


 15%|█▌        | 3/20 [03:27<19:33, 69.05s/it]

...

 65%|██████▌   | 13/20 [14:56<08:02, 68.94s/it]

Epoch 13 of 20 took 68.961s
  training loss (in-iteration):		0.415981
  train accuracy:		85.40 %
  validation accuracy:		72.44 %


 70%|███████   | 14/20 [16:05<06:53, 68.94s/it]

Epoch 14 of 20 took 68.933s
  training loss (in-iteration):		0.390263
  train accuracy:		86.48 %
  validation accuracy:		72.62 %


KeyboardInterrupt: 

In [38]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 80:
    print "Achievement unlocked: колдун 80 уровня"
else:
    print "Нужно больше магии!"

Final results:
  test accuracy:		73.68 %
Нужно больше магии!


## Заполните форму

https://goo.gl/forms/EeadABISlVmdJqgr2 